In [1]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add project root to path
sys.path.append(os.path.abspath('..'))

from src.data_loader import DataLoader
from src.validators.fx_consistency import FXConsistencyValidator

%matplotlib inline

## 1. Load Data
We load the portfolio data from the CSV file.

In [2]:
data_path = '../data/Test.csv'
loader = DataLoader(data_path)
positions, trades = loader.load_data()

print(f"Loaded {len(positions)} positions and {len(trades)} trades.")

Loaded 7721 positions and 468 trades.


## 2. FX Consistency Validation
We check for assets with the same currency having different exchange rates on the same day.

In [3]:
validator = FXConsistencyValidator(positions, trades)
errors = validator.validate()

print(f"Found {len(errors)} FX consistency errors.")

# Convert errors to DataFrame for display
if errors:
    error_df = pd.DataFrame([vars(e) for e in errors])
    cols = ['date', 'ticker', 'error_type', 'severity', 'description']
    display(error_df[cols])
else:
    print("No FX consistency errors found.")

Found 0 FX consistency errors.
No FX consistency errors found.


## 3. Visualization
Let's visualize the FX rates for currencies with errors.

In [4]:
if errors:
    # Extract currency from description or just plot all currencies with errors
    # We can find which currencies are affected by looking at the tickers in the error list
    # and mapping back to the positions dataframe.
    
    error_tickers = [e.ticker for e in errors]
    affected_currencies = positions[positions['P_Ticker'].isin(error_tickers)]['Currency'].unique()
    
    for currency in affected_currencies:
        if currency == 'USD': continue # Skip USD usually 1.0
        
        print(f"Visualizing {currency} Exchange Rates")
        
        # Get all data for this currency
        curr_data = positions[positions['Currency'] == currency].copy()
        
        plt.figure(figsize=(12, 6))
        sns.lineplot(data=curr_data, x='Date', y='Exchange Rate', hue='P_Ticker', marker='o')
        plt.title(f'{currency} Exchange Rates by Ticker')
        plt.ylabel('Exchange Rate')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.show()